# GRS Trial 1
---
## Modifying Traditional Polygenic Risk Scores

In particular, we work with Single Nucleotide Polymorphisms (SNPs) which are individual genetic variants that cause mutations and/or regulatory changes to protein expression levels, affecting phenotype. In this problem, we are interested in isolating the effect of individual SNP markers on the patient's risk of breast cancer.

Existing polygenic risk scores represent the weighted sum of all risk alleles carried by an individual. 
$$PRS=\sum_{i=1}^{29} \beta_i \cdot SNP_i$$

We are interesting in fitting models to our data in order to best evaluate a PRS score with different beta values. More formally, our lab represents this as the feature score, FS, for each beta value.

$$GRS=\sum_{i=1}^{29} FS_i \cdot \beta_i \cdot SNP_i$$

## Heuristic Identification of Biological Architectures for simulating Complex Hierarchical Interactions (HIBACHI)

HIBACHI gives us simulated data to play with. 


In [4]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import eli5
from eli5.sklearn import PermutationImportance

bcdata = pd.read_csv('~/Documents/MooreLab/VgMed/breastCancer29.csv')

Xdata = bcdata.loc[:, bcdata.columns != 'phenotype']
Ydata = bcdata['phenotype']
X_train, X_test, Y_train, Y_test = train_test_split(Xdata, Ydata, random_state=42,
                                                    train_size=0.75, test_size=0.25)

model = TPOTClassifier(generations=100, population_size=100, verbosity=2, max_time_mins=5, early_stop=5)
model = model.fit(X_train, Y_train)

Generation 1 - Current best internal CV score: 0.5649548168632541
Generation 2 - Current best internal CV score: 0.5649548168632541
Generation 3 - Current best internal CV score: 0.5649661342814681

5.06352775 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: BernoulliNB(FeatureAgglomeration(input_matrix, affinity=l1, linkage=complete), alpha=0.001, fit_prior=True)


In [10]:
# Coefficients from logit regression forms Betas for PRS function

from sklearn.linear_model import LogisticRegression
LogisticRegression(solver='lbfgs')
import warnings
warnings.filterwarnings("ignore")

# initialize list of betas to be a matrix. Each is the coefficient of the logit model, on each SNP/feature
listofbetas = []
for i in range(X_train.shape[1]):
    logit = LogisticRegression()
    logit.fit(X_train, Y_train)
    beta = logit.coef_.flatten()[i]
    listofbetas.append(beta)
    
beta_matrix = np.matrix(listofbetas).transpose()
y_matrix = np.matrix(X_test)
# beta_matrix.shape == (29,1)
# y_matrix.shape == (626, 29)
PRS = y_matrix * beta_matrix

print(beta_matrix.shape)
print(y_matrix.shape)
beta_prob_val = y_matrix * beta_matrix # np.array(beta_prob_val)[0][0]
beta_prob_val = np.array(beta_prob_val)[0][0]
beta_prob_val

(29, 1)
(626, 29)


0.03824380628908271

In [11]:
# Using Permutation Importance Scores as means to obtain GRS
# https://eli5.readthedocs.io/en/latest/blackbox/permutation_importance.html

perm = PermutationImportance(model, n_iter=100).fit(X_train, Y_train)
feat_imp = perm.feature_importances_
feat_name = X_train.columns.values
importance_df = pd.DataFrame({'feature': feat_name, 'score': feat_imp})
importance_df.sort_values('score', ascending = False)

scalar = np.matrix(feat_imp).flatten().transpose()
beta_FS = np.multiply(scalar, beta_matrix)
GRS = y_matrix * beta_FS


In [12]:
print(PRS.shape)
print(GRS.shape)

(626, 1)
(626, 1)


In [13]:
PRS_df = pd.DataFrame(PRS, columns=['PRS_score'])
GRS_df = pd.DataFrame(GRS, columns=['GRS_score'])
phenotypes = pd.DataFrame(Y_test).reset_index().rename(columns={'index':'subject_id'})
sub_ids = pd.DataFrame(X_test.index.values, columns=["subject_id"])

In [14]:
print(PRS_df.shape)
print(GRS_df.shape)
print(phenotypes.shape)
print(sub_ids.shape)

(626, 1)
(626, 1)
(626, 2)
(626, 1)


In [15]:
final_df = pd.concat([sub_ids, PRS_df, GRS_df], axis=1)
final_df = final_df.merge(phenotypes, on="subject_id")
final_df.head(5)

,subject_id,PRS_score,GRS_score,phenotype
0,1835,0.038244,-0.000106,1
1,252,0.117780,0.000089,1
2,1594,0.109034,-0.000160,0
3,430,0.075651,-0.000005,0
4,1356,0.020889,0.000016,1


In [16]:
final_df.to_csv('PRS_GRS_scores.csv', index=False)